In [25]:
import pandas as pd
pd.set_option('display.max_rows',500)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
import pandas_datareader.data as pdr
from pandas_datareader.data import DataReader
import yfinance as yf

# For time stamps
import datetime

from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [5]:
# download stocks
file = pd.read_csv('M6_Universe.csv')
stocks_list = file['symbol'].values.tolist()

end = datetime.datetime.now()
start = datetime.datetime(end.year - 3, end.month, end.day)

last_month_days = []
for year in range(start.year, end.year+1):
    for month in range(1,13):
        first_day = datetime.datetime(year, month, 1)
        prev_day = first_day - datetime.timedelta(days=1)
        last_month_days += [prev_day.strftime('%Y-%m-%d')]

## Download 100 assets

In [6]:
all_stocks = DataReader(stocks_list, 'yahoo', start, end)['Adj Close']
spy = DataReader(['SPY'], 'yahoo', start, end)['Adj Close']

marketCaps = pdr.get_quote_yahoo(stocks_list)['marketCap']
all_stocks.describe()

Symbols,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
count,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,...,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000,757.000000
mean,102.831448,259.452745,83.945543,138.864802,114.091656,95.072042,208.146701,140.154735,190.569924,158.509624,...,30.874601,113.027225,49.455466,153.092065,86.588644,63.661521,62.188460,64.971303,70.017364,64.611381
std,27.513807,62.194369,7.528617,25.592850,14.752115,36.518373,64.537268,30.645435,34.024741,40.407339,...,6.084808,16.719646,15.231962,28.605499,14.071567,11.679117,6.086397,7.228343,13.515291,45.777967
min,56.668434,138.960983,64.298126,77.114685,76.430466,37.178921,78.741379,83.830498,111.204735,76.020432,...,16.901665,72.100311,21.039654,85.628128,47.169891,39.391167,41.872684,45.845875,36.707249,17.629999
25%,79.855087,202.538116,78.960930,119.666336,100.489998,58.885761,150.231537,111.077499,157.665817,119.659935,...,25.591757,98.397064,36.365059,124.972473,75.396927,54.072262,57.728580,58.567532,57.105728,25.490000
50%,100.153282,254.167068,82.863365,134.326141,112.636421,97.610001,214.727295,154.460495,193.457581,161.869995,...,31.124092,113.148537,48.406086,155.352661,87.505814,61.171936,61.923470,64.301704,72.932571,56.360001
75%,114.312630,312.488495,87.004547,159.152878,127.577919,131.668045,265.139282,164.611496,217.928314,199.500000,...,36.595703,129.619995,55.013714,176.753571,99.967064,75.333572,66.116241,70.842850,82.360046,93.800003
max,171.877823,411.651459,104.940002,192.390640,146.209808,166.292816,324.398621,186.570496,254.599045,224.802582,...,41.039803,142.229568,91.256622,210.293747,106.228661,85.302856,77.739998,79.969498,89.927559,276.000000


In [7]:
# extract monthly returns
month_stocks = all_stocks.copy()
month_stocks['Datestring'] = month_stocks.index
    
month_stocks['year'] = pd.DatetimeIndex(month_stocks.index).year
month_stocks['month'] = pd.DatetimeIndex(month_stocks.index).month

month_on_month = month_stocks.sort_values(by='Date').groupby(['year','month']).last()
month_on_month = month_on_month.drop(month_on_month.tail(1).index).set_index(['Datestring'])

monthly_returns = month_on_month.pct_change().round(3).T
monthly_returns

Datestring,2019-08-30,2019-09-30,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31,2020-04-30,2020-05-29,...,2021-10-29,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-29,2022-05-31,2022-06-30,2022-07-29
Symbols,,,,,,,,,,,,,,,,,,,,,
ABBV,NaN,0.152,0.066,0.103,0.009,-0.073,0.058,-0.111,0.095,0.127,...,0.076,0.005,0.175,0.021,0.079,0.097,-0.086,0.003,0.039,-0.054
ACN,NaN,-0.029,-0.032,0.085,0.047,-0.022,-0.120,-0.096,0.139,0.089,...,0.125,-0.004,0.160,-0.145,-0.106,0.067,-0.107,-0.006,-0.070,0.107
AEP,NaN,0.028,0.007,-0.025,0.035,0.103,-0.138,-0.104,0.039,0.035,...,0.043,-0.034,0.098,0.016,0.012,0.101,-0.007,0.038,-0.060,0.027
AIZ,NaN,0.023,0.002,0.059,-0.013,-0.004,-0.072,-0.137,0.021,-0.028,...,0.023,-0.053,0.025,-0.021,0.117,0.071,0.000,-0.025,-0.022,0.017
ALLE,NaN,0.080,0.120,0.034,0.040,0.038,-0.111,-0.197,0.093,-0.008,...,-0.029,-0.036,0.074,-0.073,-0.067,-0.038,0.041,-0.023,-0.120,0.080
AMAT,NaN,0.039,0.087,0.071,0.054,-0.050,0.005,-0.212,0.084,0.135,...,0.062,0.079,0.069,-0.122,-0.027,-0.018,-0.163,0.065,-0.224,0.165
AMP,NaN,0.140,0.026,0.093,0.017,-0.007,-0.141,-0.275,0.122,0.229,...,0.144,-0.038,0.042,0.009,-0.011,0.002,-0.116,0.045,-0.140,0.136
AMZN,NaN,-0.023,0.023,0.014,0.026,0.087,-0.062,0.035,0.269,-0.013,...,0.027,0.040,-0.049,-0.103,0.027,0.061,-0.238,-0.033,-0.116,0.271
AVB,NaN,0.020,0.011,-0.015,-0.015,0.033,-0.074,-0.259,0.107,-0.043,...,0.076,0.009,0.064,-0.033,-0.023,0.047,-0.084,-0.086,-0.058,0.101


In [120]:
# Generate monthly return forecast table
future_steps = 4
forecast_df_dict = {}
first_time = True
for ci in [0.80, 0.60]:
    print('CI: %f' % ci)
    for ticker in stocks_list:
        print('Ticker: %s...' % ticker)
        df = monthly_returns.T[ticker].dropna().reset_index().rename(columns={'Datestring':'ds', ticker:'y'})
        model = Prophet(interval_width=ci).fit(df)
        future_dates = model.make_future_dataframe(periods=future_steps, freq='M')
        forecast = model.predict(future_dates)
        if first_time:
            forecast_df = forecast[['ds']].set_index('ds')
            lower_df = forecast_df.copy()
            yhat_df = forecast_df.copy()
            upper_df = forecast_df.copy()
            
            first_time = False
            
        forecast = forecast.set_index('ds')
        
        lower_df[ticker] = forecast['yhat_lower']
        yhat_df[ticker] = forecast['yhat']
        upper_df[ticker] = forecast['yhat_upper']
        forecast_df_dict[ci] = (lower_df, yhat_df, upper_df)
        

01:33:31 - cmdstanpy - INFO - Chain [1] start processing
01:33:31 - cmdstanpy - INFO - Chain [1] done processing


CI: 0.800000
Ticker: ABBV...


01:33:32 - cmdstanpy - INFO - Chain [1] start processing
01:33:32 - cmdstanpy - INFO - Chain [1] done processing


Ticker: ACN...


01:33:34 - cmdstanpy - INFO - Chain [1] start processing


...

Ticker: XLP...


01:38:18 - cmdstanpy - INFO - Chain [1] start processing
01:38:19 - cmdstanpy - INFO - Chain [1] done processing


Ticker: XLB...


01:38:20 - cmdstanpy - INFO - Chain [1] start processing
01:38:20 - cmdstanpy - INFO - Chain [1] done processing


Ticker: VXX...


In [121]:
forecast_df_dict[0.8][0]

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
ds,,,,,,,,,,,,,,,,,,,,,
2019-09-30,0.075099,-0.066896,-0.055474,-0.019061,0.016709,-0.020905,0.057154,-0.054826,-0.025809,-0.068977,...,-0.004977,-0.031658,-0.040661,-0.012254,-0.010490,-0.020753,-0.013654,-0.006812,-0.020232,-0.214880
2019-10-31,-0.008392,-0.066079,0.018404,-0.026802,0.032125,0.019246,0.001955,-0.019473,-0.071528,0.081895,...,-0.025874,-0.007875,-0.105679,-0.038108,-0.040202,-0.002569,-0.008184,-0.042321,-0.028525,-0.211266
2019-11-29,0.035462,0.025701,-0.073152,0.018790,-0.024764,-0.021658,0.004957,-0.031791,-0.077655,-0.023648,...,-0.016907,0.015102,-0.120103,-0.036780,-0.014296,-0.005204,-0.058006,-0.015675,-0.021530,-0.218147
2019-12-31,-0.022322,-0.006940,-0.014088,-0.057567,-0.014967,-0.061269,-0.105675,-0.035436,-0.088206,-0.035315,...,-0.059013,0.008455,-0.141144,-0.043198,-0.059849,-0.024317,-0.012048,0.014734,-0.036323,-0.106226
2020-01-31,-0.054923,-0.093046,-0.005814,-0.051780,-0.053846,-0.057596,-0.007783,0.000684,-0.044819,-0.055115,...,-0.046459,-0.048450,-0.046121,-0.033100,-0.056559,-0.016170,-0.021515,-0.036189,-0.070258,0.009009
2020-02-28,0.033972,-0.091791,-0.103871,-0.035447,-0.092552,0.029549,-0.055827,-0.004180,-0.064429,-0.106487,...,-0.057693,-0.042935,-0.087313,-0.044804,-0.051677,-0.035842,-0.086029,-0.062002,-0.041668,0.025680
2020-03-31,-0.109780,-0.042847,-0.052874,-0.073335,-0.098579,-0.114741,-0.198247,-0.056079,-0.152191,-0.055362,...,-0.145582,-0.020921,-0.280200,-0.093018,-0.106028,-0.082377,-0.045987,-0.023895,-0.080846,0.306678
2020-04-30,0.056313,0.092720,0.011715,0.032634,0.056367,0.028363,0.097286,0.202260,0.063307,0.103756,...,0.063819,0.083365,0.109485,0.140410,0.050508,0.113392,0.022588,0.030581,0.099040,-0.375645
2020-05-29,0.019386,0.013697,-0.028046,-0.041476,-0.014637,0.075204,0.079132,-0.023630,-0.053289,-0.023531,...,0.002253,0.007915,-0.014046,-0.005069,0.003699,0.035321,-0.011307,-0.019911,0.034512,-0.269794


In [123]:
# Quantile table generation
timestamp = '2022-08-31'
quantile_table = pd.DataFrame([],columns=stocks_list).T
quantile_table['20'] = np.nan
quantile_table['40'] = np.nan
quantile_table['y'] = np.nan
quantile_table['60'] = np.nan
quantile_table['80'] = np.nan

quantile_table = quantile_table.T

for ticker in stocks_list:
    q_20 = forecast_df_dict[0.8][0][ticker][timestamp]
    q_40 = forecast_df_dict[0.6][0][ticker][timestamp]
    q_y = forecast_df_dict[0.8][1][ticker][timestamp]
    q_60 = forecast_df_dict[0.6][2][ticker][timestamp]
    q_80 = forecast_df_dict[0.8][2][ticker][timestamp]

    quantile_table[ticker] = [q_20, q_40, q_y, q_60, q_80]
    
    
quantile_table = quantile_table.T
quantile_table

,20,40,y,60,80
ABBV,-0.097098,-0.097098,-0.054045,-0.010630,-0.010630
ACN,0.032675,0.032675,0.062696,0.093324,0.093324
AEP,0.004890,0.004890,0.035959,0.071027,0.071027
AIZ,0.084970,0.084970,0.117144,0.145739,0.145739
ALLE,0.018858,0.018858,0.058916,0.101185,0.101185
AMAT,0.000214,0.000214,0.049323,0.099380,0.099380
AMP,0.006621,0.006621,0.055314,0.101953,0.101953
AMZN,0.009675,0.009675,0.048506,0.087066,0.087066
AVB,-0.001407,-0.001407,0.037606,0.077950,0.077950
AVY,0.125895,0.125895,0.155462,0.189009,0.189009


In [124]:
# predicted return ranking algorithm
rank1_indices = quantile_table['20'].sort_values(ascending=True).index[:20]
rank2_indices = quantile_table['40'].sort_values(ascending=True).index[20:40]
rank3_indices = quantile_table['y'].sort_values(ascending=True).index[40:60]
rank4_indices = quantile_table['60'].sort_values(ascending=True).index[60:80]
rank5_indices = quantile_table['80'].sort_values(ascending=True).index[80:]

rank_df = quantile_table.copy()
rank_df[:] = 0

rank_df.loc[rank1_indices,'20'] = 1
rank_df.loc[rank2_indices,'40'] = 1
rank_df.loc[rank3_indices,'y'] = 1
rank_df.loc[rank4_indices,'60'] = 1
rank_df.loc[rank5_indices,'80'] = 1

# Set empty rows to carry rank 3
empty_rows = (rank_df.sum(axis=1) == 0)
rank_df.loc[empty_rows, :] = 0.2

rank_df = rank_df.T

# normalize rows
rank_df = rank_df / rank_df.sum()
rank_df = rank_df.T.round(4).rename(columns={'20':'Rank 1','40':'Rank 2','y':'Rank 3','60':'Rank 4','80':'Rank 5'})
rank_df

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5
ABBV,1.0,0.0000,0.0000,0.0000,0.0
ACN,0.0,0.0000,0.0000,1.0000,0.0
AEP,0.0,1.0000,0.0000,0.0000,0.0
AIZ,0.0,0.0000,0.0000,0.0000,1.0
ALLE,0.0,0.0000,0.0000,1.0000,0.0
AMAT,0.0,0.3333,0.3333,0.3333,0.0
AMP,0.0,0.3333,0.3333,0.3333,0.0
AMZN,0.0,0.0000,1.0000,0.0000,0.0
AVB,0.0,0.5000,0.5000,0.0000,0.0
AVY,0.0,0.0000,0.0000,0.0000,1.0


In [125]:
#portfolio strategy
strategy = rank_df['Rank 5'] / rank_df['Rank 5'].sum()
strategy = strategy.round(5)
strategy

ABBV      0.00000
ACN       0.00000
AEP       0.00000
AIZ       0.04808
ALLE      0.00000
AMAT      0.00000
AMP       0.00000
AMZN      0.00000
AVB       0.00000
AVY       0.04808
AXP       0.00000
BDX       0.00000
BF-B      0.00000
BMY       0.00000
BR        0.00000
CARR      0.00962
CDW       0.04808
CE        0.04808
CHTR      0.04808
CNC       0.00000
CNP       0.00000
COP       0.02404
CTAS      0.04808
CZR       0.04808
DG        0.00000
DPZ       0.00000
DRE       0.00000
DXC       0.04808
META      0.04808
FTV       0.00000
GOOG      0.00000
GPC       0.00000
HIG       0.04808
HST       0.04808
JPM       0.00000
KR        0.00000
OGN       0.00000
PG        0.00000
PPL       0.00000
PRU       0.04808
PYPL      0.04808
RE        0.00000
ROL       0.00000
ROST      0.00000
UNH       0.00000
URI       0.04808
V         0.00000
VRSK      0.00000
WRK       0.04808
XOM       0.00000
IVV       0.00000
IWM       0.00000
EWU       0.00000
EWG       0.00000
EWL       0.00000
EWQ       

In [126]:
# Save to file
submission = rank_df.copy()

diff = 1 - submission[['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5']].sum(1)
submission['Rank 5'] = submission['Rank 5'] + diff

diff = 1 - strategy.sum()
strategy[-1] = strategy[-1] + diff

submission['Decision'] = strategy
submission = submission.round(5)
submission.to_csv('submission.csv')
submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,1.0,0.0000,0.0000,0.0000,0.0000,0.00000
ACN,0.0,0.0000,0.0000,1.0000,0.0000,0.00000
AEP,0.0,1.0000,0.0000,0.0000,0.0000,0.00000
AIZ,0.0,0.0000,0.0000,0.0000,1.0000,0.04808
ALLE,0.0,0.0000,0.0000,1.0000,0.0000,0.00000
AMAT,0.0,0.3333,0.3333,0.3333,0.0001,0.00000
AMP,0.0,0.3333,0.3333,0.3333,0.0001,0.00000
AMZN,0.0,0.0000,1.0000,0.0000,0.0000,0.00000
AVB,0.0,0.5000,0.5000,0.0000,0.0000,0.00000
AVY,0.0,0.0000,0.0000,0.0000,1.0000,0.04808
